## Price_Indicator_Analysis_MVP

Analysis of common technical analysis indicators on daily price data for different products.

### Data Collection

In [1]:
import pandas as pd
import numpy as np

import requests
import json
import quandl

import datetime

import matplotlib.pyplot as plt

import sqlite3

% matplotlib inline

The first step will be to get data from the [quandl API](https://www.quandl.com/collections/futures).  For this we use an api key generated from signing up for a free quandl account.

We create a list of the symbols for the products we are interested in, which can be found by looking at the [documentation](https://docs.quandl.com/).

Then loop through the products, read the price data into a dataframe, and attach the symbol:dataframe key:value pair to a dictionary object, after selecting the specific information we need.

In [3]:
# Get data from Quandl with api key
api_key = 'Hv95pPh1xQWzt5DFhxS7'

# Create list of symbols for the products we want data for
prod_list = ['CL', 'HO', 'NG', 'GC',
             'SI', 'AD', 'CD', 'EC',
             'BP', 'JY', 'US', 'C',
             'W', 'S', 'ES']

# Create a dict with keys as symbols and values as dataframe of price info
prod_dict = {}

# Iterate through list of prods and add data to dict
for prod in prod_list:
    # Quandl API call
    df = quandl.get('SCF/CME_{p}1_FW'.format(p=prod), authtoken=api_key)
    
    # Drop open interest column and rename Settle column
    df.drop(['Prev. Day Open Interest'], axis=1, inplace=True)
    df.rename(columns={'Open': 'open',
                      'High': 'high',
                      'Low': 'low',
                      'Settle': 'close',
                      'Volume': 'volume'}, inplace=True)
    # Add prod to dict
    prod_dict[prod] = df

Here is some summary information of the Crude Oil ('CL') dataframe

In [6]:
cl = prod_dict['CL']
cl.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2517 entries, 2005-01-03 to 2014-12-31
Data columns (total 5 columns):
open      2517 non-null float64
high      2517 non-null float64
low       2517 non-null float64
close     2517 non-null float64
volume    2517 non-null float64
dtypes: float64(5)
memory usage: 118.0 KB


In [7]:
cl.describe().T

,count,mean,std,min,25%,50%,75%,max
open,2517.0,81.796207,19.832869,34.36,65.68,82.55,96.90,145.19
high,2517.0,82.893532,19.961713,36.25,66.56,83.72,97.82,147.27
low,2517.0,80.582003,19.695067,33.55,64.65,81.25,95.56,143.22
close,2517.0,81.779758,19.855341,33.98,65.82,82.55,96.92,145.29
volume,2517.0,241235.096146,96750.320207,50.00,175510.60,242868.00,301777.00,670082.00


In [8]:
cl.head()

,open,high,low,close,volume
Date,,,,,
2005-01-03,43.10,43.10,41.25,42.12,69484.0
2005-01-04,42.18,44.13,41.85,43.91,100665.0
2005-01-05,43.90,44.00,42.60,43.39,91011.0
2005-01-06,43.39,45.75,42.80,45.56,122411.0
2005-01-07,45.50,46.10,44.75,45.43,99725.0


Now we have the data from quandl broken down into specific products and we have the columns we are interested in as well as a datetime object as an index.  Next we will do the same thing with the data from the [Cryptocompare API](https://www.cryptocompare.com/api/).

First step here will be do define the url and parameters we need.  This information can be gathered by reading through the documentation.  

Again we need to create a list of symbol names to use as arguments when calling the API.

Then we do a `get` request using the `requests` library, loop through the symbols list and do some manipulation to the final dataframe.  

When this is all done we can just append the df to our prod_dict to put them all together.

In [12]:
# Cryptocompare API url
url = 'https://min-api.cryptocompare.com/data/histoday'

# List of cryptocurrencies to gather data for
symbol_list = ['BTC', 'ETH', 'XRP', 'LTC', 'XLM',
               'EOS', 'NEO', 'ADA', 'BCH', 'XMR']

# Iterate through the list to create a df for each symbol
for symbol in symbol_list:
    # Set relevant parameters, currency in USD and limit of 2000 data points
    params = {'fsym': symbol, 'tsym': 'USD', 'limit': 2000}
    
    # Call API and put data into a dataframe
    response = requests.get(url, params=params)
    data = response.json()['Data']
    df = pd.DataFrame(data)
    
    # Add date column and set to index
    df['Date'] = [datetime.date.fromtimestamp(d) for d in df.time]
    df = df[['open', 'high', 'low', 'close', 'volumeto', 'Date']]
    df.set_index('Date', inplace=True)
    
    # Rename volumeto column
    df.rename(columns={'volumeto': 'volume'}, inplace=True)
    
    # Append to prod_dict
    prod_dict[symbol] = df


Now all the products we are interested in are in the dictionary.

In [15]:
prod_dict.keys()

dict_keys(['CL', 'HO', 'NG', 'GC', 'SI', 'AD', 'CD', 'EC', 'BP', 'JY', 'US', 'C', 'W', 'S', 'ES', 'BTC', 'ETH', 'XRP', 'LTC', 'XLM', 'EOS', 'NEO', 'ADA', 'BCH', 'XMR'])

And just like before we can take a look at one of the new products to see that the information is in the same format.  Lets look at Bitcoin.

In [16]:
btc = prod_dict['BTC']
btc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2001 entries, 2012-10-08 to 2018-03-31
Data columns (total 5 columns):
open      2001 non-null float64
high      2001 non-null float64
low       2001 non-null float64
close     2001 non-null float64
volume    2001 non-null float64
dtypes: float64(5)
memory usage: 93.8+ KB


In [17]:
btc.describe().T

,count,mean,std,min,25%,50%,75%,max
open,2001.0,1.483754e+03,2.989586e+03,10.17,235.82,442.63,893.75,1.934660e+04
high,2001.0,1.543917e+03,3.128966e+03,10.65,240.00,452.64,919.32,1.987062e+04
low,2001.0,1.413044e+03,2.807586e+03,9.74,231.73,430.89,845.87,1.875091e+04
close,2001.0,1.487144e+03,2.991748e+03,10.17,235.83,442.66,894.16,1.934549e+04
volume,2001.0,1.691150e+08,4.809417e+08,109479.89,3945752.94,15315755.66,47619189.45,6.245732e+09


In [18]:
btc.head()

,open,high,low,close,volume
Date,,,,,
2012-10-08,11.78,12.35,11.57,11.90,559694.91
2012-10-09,11.90,12.19,11.77,12.12,333351.59
2012-10-10,12.12,12.15,11.86,12.03,238483.07
2012-10-11,12.03,12.15,11.92,12.00,259522.53
2012-10-12,12.00,12.11,11.85,11.86,141870.31


With all the data now collected and stored in a dictionary we want to create a SQL schema to store the data locally.  I chose to use a **3 table schema** with one table, **Data**, keeping track of my data sources, **Symbols** housing all the product symbols with some product specific information, and finally **Daily_Prices** which holds all the daily price and volume information.  

The **Symbols** table links to the **Data** table on an id and the **Daily_Prices** table links to the **Symbols** table on the symbol name.

Here is the creation of the three tables, starting with **Data**

In [19]:
# SQLite file name to store database
sqlite_file = 'securities_master_db.sqlite'

# DATA TABLE
table_name = 'Data'
id_col = 'id'
name_col = 'name'
url_col = 'url'
dtype_int = 'INTEGER'
dtype_text = 'TEXT'

# Connect to the database file
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

# Create a new table with 3 columns
c.execute('CREATE TABLE {tn} ({ic} {dti} PRIMARY KEY, {nc} {dtt}, {uc} {dtt})'\
          .format(tn=table_name, ic=id_col, dti=dtype_int, nc=name_col, dtt=dtype_text, uc=url_col))

# Commit changes and close
conn.commit()
conn.close()

With the table created, I chose to manually add the data sources since there are only three.

In [20]:
# Connect to database file
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

# Add value for Cryptocompare to Data table
c.execute("INSERT INTO {tn} ({ic}, {nc}, {uc}) VALUES (1, 'Cryptocompare', 'https://min-api.cryptocompare.com')"\
          .format(tn=table_name, ic=id_col, nc=name_col, uc=url_col))

# Add value for Quandl to Data table
c.execute("INSERT INTO {tn} ({ic}, {nc}, {uc}) VALUES (2, 'Quandl', 'https://docs.quandl.com')"\
          .format(tn=table_name, ic=id_col, nc=name_col, uc=url_col))

# Add value for Quantopian to Data table
c.execute("INSERT INTO {tn} ({ic}, {nc}, {uc}) VALUES (3, 'Quantopian', 'https://www.quantopian.com/data')"\
          .format(tn=table_name, ic=id_col, nc=name_col, uc=url_col))

# Commit and close
conn.commit()
conn.close()

Now we will add the **Symbols** table

In [21]:
# SYMBOLS TABLE
# Initialize variables for file name, table, columns, data types
table_name = 'Symbols'

data_table = 'Data'
data_id = 'id'

id_col = 'id'
data_id_col = 'data_id'
symbol_col = 'symbol'
name_col = 'name'
sector_col = 'sector'
exchange_col = 'exchange'
dtype_int = 'INTEGER'
dtype_text = 'TEXT'

# Connect to the database file
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

# Create a new table with 3 columns
c.execute('CREATE TABLE {tn} ({ic} {dti} PRIMARY KEY,\
                              {dc} {dti},\
                              {sc} {dtt},\
                              {nc} {dtt},\
                              {sec} {dtt},\
                              {ec} {dtt},\
                              FOREIGN KEY ({dc}) REFERENCES {dt} ({dic}))'\
         .format(tn=table_name, ic=id_col, dti=dtype_int, dc=data_id_col, sc=symbol_col,\
                 dtt=dtype_text, nc=name_col, sec=sector_col, ec=exchange_col,\
                 dt=data_table, dic=data_id))

# Commit changes and close
conn.commit()
conn.close()

Now that we have the **Symbols** table we can insert the information.  I chose to add some information about each product that I thought would be interesting for later analysis.  The following dictionary maps all the *products* to a `list` of information about that product including *data source, name, sector, exchange*.

In [23]:
# Dict of all products with maps to data_id, name, sector, and exchange
products = {'CL': [2, 'Crude', 'Energy', 'CME'],
            'HO': [2, 'HeatOil', 'Energy', 'CME'],
            'NG': [2, 'NatGas', 'Energy', 'CME'],
            'GC': [2, 'Gold', 'Metals', 'CME'],
            'SI': [2, 'Silver', 'Metals', 'CME'],
            'AD': [2, 'Aussie', 'Forex', 'CME'],
            'CD': [2, 'Canadien', 'Forex', 'CME'],
            'EC': [2, 'Euro', 'Forex', 'CME'],
            'BP': [2, 'Pound', 'Forex', 'CME'],
            'JY': [2, 'Yen', 'Forex', 'CME'],
            'US': [2, '30-yr', 'Treasuries', 'CME'],
            'C': [2, 'Corn', 'Grains', 'CME'],
            'W': [2, 'Wheat', 'Grains', 'CME'],
            'S': [2, 'Soybeans', 'Grains', 'CME'],
            'ES': [2, 'E-mini', 'Indexes', 'CME'],
            'BTC': [1, 'Bitcoin', 'Cryptocurrency', 'CCAgg'],
            'ETH': [1, 'Ethereum', 'Cryptocurrency', 'CCAgg'],
            'XRP': [1, 'Ripple', 'Cryptocurrency', 'CCAgg'],
            'BCH': [1, 'BitcoinCash', 'Cryptocurrency', 'CCAgg'],
            'LTC': [1, 'Litecoin', 'Cryptocurrency', 'CCAgg'],
            'ADA': [1, 'Cardano', 'Cryptocurrency', 'CCAgg'],
            'NEO': [1, 'Neo', 'Cryptocurrency', 'CCAgg'],
            'XLM': [1, 'Stellar', 'Cryptocurrency', 'CCAgg'],
            'EOS': [1, 'EOS', 'Cryptocurrency', 'CCAgg'],
            'XMR': [1, 'Monero', 'Cryptocurrency', 'CCAgg'],}

Here is the code to insert the information into the **Symbols** table

In [25]:
# Create the column name list for database insertion
table_name = 'Symbols'
cols = ['data_id', 'symbol', 'name', 'sector', 'exchange']

# Open a connection to the database
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

# Iterate through all symbols of product_dict
for symbol, s_info in products.items():
    # Set params and insert row into database
    params = (s_info[0], symbol, s_info[1], s_info[2], s_info[3])
    c.execute("INSERT INTO {tn} ({c0}, {c1}, {c2}, {c3}, {c4}) VALUES (?, ?, ?, ?, ?)"\
            .format(tn=table_name, c0=cols[0], c1=cols[1], c2=cols[2],\
            c3=cols[3], c4=cols[4]), params)
    
# Close connection to database
conn.commit()
conn.close()

And finally the **Daily_Prices** table

In [22]:
# DAILY_PRICES TABLE
# Initialize variables for file name, table, columns, data types
table_name = 'Daily_Prices'
symbols_table = 'Symbols'

id_col = 'id'
data_id_col = 'data_id'
symbol_col = 'symbol'
date_col = 'date'
open_col = 'open'
high_col = 'high'
low_col = 'low'
close_col = 'close'
volume_col = 'volume'

dtype_int = 'INTEGER'
dtype_text = 'TEXT'
dtype_real = 'REAL'

# Connect to the database file
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

# Create a new table with 3 columns
c.execute('CREATE TABLE {tn} ({ic} {dti} PRIMARY KEY,\
                              {dc} {dti},\
                              {sc} {dtt},\
                              {dtc} {dtt},\
                              {oc} {dtr},\
                              {hc} {dtr},\
                              {lc} {dtr},\
                              {cc} {dtr},\
                              {vc} {dti},\
                              FOREIGN KEY ({sc}) REFERENCES {st} ({sc}))'\
         .format(tn=table_name, ic=id_col, dti=dtype_int, dc=data_id_col, sc=symbol_col,\
                 dtt=dtype_text, dtc=date_col, oc=open_col, dtr=dtype_real, hc=high_col,\
                 lc=low_col, cc=close_col, vc=volume_col, st=symbols_table))

# Commit changes and close
conn.commit()
conn.close()

We will hold off on inserting the price data into the table right now because it needs to be cleane